In [1]:
import os
import pickle

data_folder = '/home/mcwave/code/automath/atp/datasets/provability/mathlib4_states_w_proof/'
file_names = os.listdir(data_folder)

data = []

count = 0
for file_name in file_names:
    if not file_name.endswith("pkl"):
        continue
    if not 'Algebra' in file_name:
        continue
    count += 1
    if count <= 5:
        continue
    print("Loading", file_name)
    file_path = os.path.join(data_folder, file_name)
    fin = open(file_path, 'rb')
    while True:
        try:
            pair = pickle.load(fin)
            data.append(pair) #(pair[1][0], pair[1][2][0]))
        except:
            break
    break

print(len(data), "examples loaded")

Loading Mathlib__Algebra__Ring__Subring__Pointwise.lean.pkl
12680 examples loaded


In [2]:
data[1]

(('https://github.com/leanprover-community/mathlib4',
  '27c6744e1c0e25d676be5eb252cd4b6d30c6acc7',
  'Mathlib/Algebra/Ring/Subring/Pointwise.lean',
  'Subring.pointwise_smul_def'),
 ('case h\nM : Type u_1\nR : Type u_2\ninst✝² : Monoid M\ninst✝¹ : Ring R\ninst✝ : MulSemiringAction M R\na : M\nS : Subring R\nnvar0 : R\n⊢ nvar0 ∈ SMul.smul a S ↔ nvar0 ∈ map (MulSemiringAction.toRingHom M R a) S',
  2,
  ['simp_rw [mem_map]', 'exact']))

In [3]:
from utils.lean_math_utils import *
from utils.lean_theorem_utils import *

def count_lines(string):
    # Split the string into lines
    lines = string.splitlines()
    # Count the number of lines
    return len(lines)

def extract_first_case(state_pp):
    state_pp = state_pp.strip()
    if not state_pp.startswith('case'):
        return state_pp
    lines = state_pp.split('\n')
    first_case = []
    for line in lines[1:]:
        if line.strip().startswith('case'):
            break
        if line.strip() != '':
            first_case.append(line)
    return '\n'.join(first_case)


# Params:
#   hyp: tuple(name, type)
#   tactics: list(tactic)
def is_hypothesis_useful(hyp, tactics):
    for tactic in tactics:
        tokens = tokenize_lean_tactic(tactic)
        if hyp[0] in tokens:
            idx = tokens.index(hyp[0])
            if idx > 0:
                if hyp[0].startswith('h'):
                    return True
                if tokens[idx - 1] == 'exact':
                    return True
                if tokens[idx - 1] == 'at':
                    return True
                if tokens[idx - 1] == '[':
                    return True
                if idx < len(tokens) - 1 and tokens[idx + 1] == ']':
                    return True
                for operator in TargetNode.operators:
                    if operator in hyp[1]:
                        return True
    return False

def create_hypothesis_predict_data(raw_state_pp, tactics, theorem_name):
    is_case = raw_state_pp.strip().startswith('case')
    state_pp = extract_first_case(raw_state_pp)
    if is_case and count_lines(state_pp) < count_lines(raw_state_pp) - 2:
        tactics = tactics[0:1]
    #
    premise = Premise()
    premise.theorem_name = theorem_name
    premise.parse_state(state_pp)
    #
    useful_hypotheses, useless_hypotheses = [], OrderedDict()
    for hyp in premise.hypotheses.items():
        useful = is_hypothesis_useful(hyp, tactics)
        if useful:
            #print("YES:", hyp)
            useful_hypotheses.append(hyp)
        else:
            #print("NO :", hyp)
            useless_hypotheses[hyp[0]] = hyp[1]
    premise.hypotheses = useless_hypotheses
    return premise, useful_hypotheses

idx = 120

state_pp = data[idx][1][0]
tactics = data[idx][1][2]
theorem_name = data[idx][0][3]

print("STATE_PP:\n" + state_pp)

print("TACTICS:\n" + "\n".join(tactics))

premise, useful_hypotheses = create_hypothesis_predict_data(state_pp, tactics, theorem_name)

print("STATE_PP:\n" + premise.to_theorem_code())
print("\nHYPOTHESES:\n", useful_hypotheses)


STATE_PP:
case h
M : Type u_1
R : Type u_2
inst✝² : Monoid M
inst✝¹ : Ring R
inst✝ : MulSemiringAction M R
a : M
S : Subring R
nvar0 : R
⊢ nvar0 ∈ map (MulSemiringAction.toRingHom M R a) S ↔ nvar0 ∈ a • S
TACTICS:
symm
simp [eq_comm (a := a)]
cases S
rw [smul_neg]
rw [← eq_f₀']
STATE_PP:
theorem Subring.pointwise_smul_def (M: Type u_1) (R: Type u_2) (inst✝²: Monoid M) (inst✝¹: Ring R) (inst✝: MulSemiringAction M R) (a: M) (S: Subring R) (nvar0: R) : nvar0 ∈ map (MulSemiringAction.toRingHom M R a) S ↔ nvar0 ∈ a • S :=

HYPOTHESES:
 []


In [3]:
# import os
# import pickle

# data_folder = '/home/mcwave/code/axiomatization/datasets/mathlib4_states_w_proof/'
# file_names = os.listdir(data_folder)

# fout = open('/home/mcwave/code/axiomatization/datasets/mathlib4_states_w_proof_hyp_pred.pkl', 'wb')

# count = 0
# for file_name in file_names:
#     if not file_name.endswith("pkl"):
#         continue
#     count += 1
#     data = []
#     print("Loading", file_name)
#     file_path = os.path.join(data_folder, file_name)
#     fin = open(file_path, 'rb')
#     while True:
#         try:
#             pair = pickle.load(fin)
#             data.append(pair) #(pair[1][0], pair[1][2][0]))
#         except:
#             break
#     #
#     fin.close()
#     print(len(data), "examples loaded")
#     hyp_data = []
#     for pair in data:
#         state_pp = pair[1][0]
#         tactics = pair[1][2]
#         if tactics is None or len(tactics) == 0:
#             continue
#         full_path = pair[0][2]
#         theorem_name = pair[0][3]
#         premise, useful_hypotheses = create_hypothesis_predict_data(state_pp, tactics, theorem_name)
#         if len(useful_hypotheses) == 0:
#             continue
#         premise.full_path = full_path
#         hyp_data.append((premise, useful_hypotheses))
#         pickle.dump((premise, useful_hypotheses), fout)
#     #
#     fout.flush()
#     print(len(hyp_data), "hypotheses data found")

# fout.close()

Loading Mathlib__Algebra__Category__ModuleCat__Presheaf.lean.pkl
50933 examples loaded
10302 hypotheses data found
Loading Mathlib__Algebra__BigOperators__Pi.lean.pkl
31644 examples loaded
7145 hypotheses data found
Loading Mathlib__Algebra__Category__ModuleCat__EpiMono.lean.pkl
20 examples loaded
0 hypotheses data found
Loading Mathlib__Algebra__DirectSum__Decomposition.lean.pkl
55012 examples loaded
14902 hypotheses data found
Loading Mathlib__Algebra__Category__GroupCat__Biproducts.lean.pkl
10067 examples loaded
326 hypotheses data found
Loading Mathlib__Algebra__Category__FGModuleCat__Basic.lean.pkl
57302 examples loaded
20074 hypotheses data found
Loading Mathlib__Algebra__Group__Basic.lean.pkl
1432363 examples loaded
345282 hypotheses data found
Loading Mathlib__Algebra__Algebra__Hom.lean.pkl
310093 examples loaded
50818 hypotheses data found
Loading Mathlib__Algebra__BigOperators__Fin.lean.pkl
37812 examples loaded
10195 hypotheses data found
Loading Mathlib__Algebra__BigOperato

8837 hypotheses data found
Loading Mathlib__Algebra__Algebra__Subalgebra__Directed.lean.pkl
15341 examples loaded
9637 hypotheses data found
Loading Mathlib__Algebra__FreeAlgebra.lean.pkl
18622 examples loaded
3703 hypotheses data found
Loading Mathlib__Algebra__Category__ModuleCat__Presheaf__Sheafify.lean.pkl
106 examples loaded
26 hypotheses data found
Loading Mathlib__Algebra__Category__ModuleCat__Algebra.lean.pkl
218 examples loaded
21 hypotheses data found
Loading Mathlib__Algebra__Algebra__Basic.lean.pkl
173728 examples loaded
13205 hypotheses data found
Loading Mathlib__Algebra__BigOperators__Finprod.lean.pkl
46121 examples loaded
18646 hypotheses data found
Loading Mathlib__Algebra__Algebra__NonUnitalHom.lean.pkl
260187 examples loaded
42581 hypotheses data found
Loading Mathlib__Algebra__CharP__MixedCharZero.lean.pkl
2351 examples loaded
876 hypotheses data found
Loading Mathlib__Algebra__DirectSum__Finsupp.lean.pkl
3352 examples loaded
101 hypotheses data found
Loading Mathli

542499 examples loaded
166521 hypotheses data found
Loading Mathlib__Algebra__Category__MonCat__FilteredColimits.lean.pkl
56 examples loaded
24 hypotheses data found
Loading Mathlib__Algebra__GCDMonoid__Multiset.lean.pkl
20993 examples loaded
1760 hypotheses data found
Loading Mathlib__Algebra__ContinuedFractions__Computation__ApproximationCorollaries.lean.pkl
42 examples loaded
8 hypotheses data found
Loading Mathlib__Algebra__Group__Pi__Basic.lean.pkl
240192 examples loaded
43047 hypotheses data found
Loading Mathlib__Algebra__Category__ModuleCat__Monoidal__Basic.lean.pkl
28095 examples loaded
5460 hypotheses data found
Loading Mathlib__Algebra__Category__GroupCat__Colimits.lean.pkl
9575 examples loaded
1899 hypotheses data found
Loading Mathlib__Algebra__Group__Prod.lean.pkl
452050 examples loaded
52490 hypotheses data found
Loading Mathlib__Algebra__Category__ModuleCat__Products.lean.pkl
728 examples loaded
0 hypotheses data found
Loading Mathlib__Algebra__BigOperators__Group__List

In [4]:
from datasets import Dataset

data = []
fin = open('/home/mcwave/code/axiomatization/datasets/mathlib4_algebra_states_w_proof_hyp_pred.pkl', 'rb')

while True:
    try:
        premise, hypotheses = pickle.load(fin)
        state_pp = premise.to_theorem_code()
        target_hyp = str([x[1] for x in hypotheses])
        data.append((state_pp, target_hyp))
    except:
        break
    
fin.close()

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW
from tqdm import tqdm
import os

class InstructionResponseDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        instruction, response = self.data[idx]
        
        input_encoding = self.tokenizer(instruction, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        output_encoding = self.tokenizer(response, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")

        input_mask = input_encoding["attention_mask"].squeeze()

        return {
            "input_ids": input_encoding["input_ids"].squeeze(),
            "attention_mask": input_mask,
            "labels": output_encoding["input_ids"].squeeze()
        }

def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Initialize tokenizer and model
model_name = "google/byt5-small"  # You can change this to any other Seq2Seq model
tokenizer = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


# Create full dataset
full_dataset = InstructionResponseDataset(data, tokenizer)

# Split into train and test sets
train_size = int(0.99 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=20, shuffle=False)

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/mcwave/.local/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/mcwave/anaconda3/envs/atp/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(384, 1472)
  (encoder): T5Stack(
    (embed_tokens): Embedding(384, 1472)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1472, out_features=384, bias=False)
              (k): Linear(in_features=1472, out_features=384, bias=False)
              (v): Linear(in_features=1472, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=1472, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1472, out_features=3584, bias=False)
              (wi_1): Linear(in_features=1472, out_features=3584, bias=False)
              (w

In [ ]:
num_epochs = 4
eval_steps = 10000

for epoch in range(num_epochs):
    num_steps = 0
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        num_steps += 1
        if num_steps > 0 and num_steps % eval_steps == 0:
            avg_train_loss = total_loss / len(train_dataloader)
            avg_test_loss = evaluate(model, test_dataloader, device)
            print(f"Epoch {epoch + 1}/{num_epochs}, steps {num_steps}")
            print(f"Average Train Loss: {avg_train_loss:.4f}")
            print(f"Average Test Loss: {avg_test_loss:.4f}")
            # Save the model after each epoch
            save_path = f"/home/mcwave/code/axiomatization/datasets/hyp_pred_byt5_small/trained_model_epoch_{epoch + 1}_steps_{num_steps}"
            os.makedirs(save_path, exist_ok=True)
            model.save_pretrained(save_path)
    

Epoch 1/4:   7%|█████████▏                                                                                                                                | 9999/150664 [3:49:47<54:04:35,  1.38s/it]

Epoch 1/4, steps 10000
Average Train Loss: 0.0322
Average Test Loss: 0.0192
[2024-08-16 16:08:48,343] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/home/mcwave/anaconda3/envs/atp/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.1
 [WARNING]  using untested triton version (2.1.0), only 1.0.0 is known to be compatible


Epoch 1/4:  13%|██████████████████▏                                                                                                                      | 19999/150664 [7:43:53<48:09:51,  1.33s/it]

Epoch 1/4, steps 20000
Average Train Loss: 0.0327
Average Test Loss: 0.0055


Epoch 1/4:  16%|██████████████████████▏                                                                                                                  | 24432/150664 [9:34:16<46:22:40,  1.32s/it]

In [4]:
from datasets import Dataset

state_pps = []
target_hyps = []
fin = open('/home/mcwave/code/axiomatization/datasets/mathlib4_states_w_proof_hyp_pred.pkl', 'rb')

while True:
    try:
        premise, hypotheses = pickle.load(fin)
        state_pp = premise.to_theorem_code()
        target_hyp = str([x[1] for x in hypotheses])
        #data.append((state_pp, target_hyp))
        state_pps.append(state_pp)
        target_hyps.append(target_hyp)
    except:
        break
    
fin.close()

ds = Dataset.from_dict({'state_pp': state_pps, 'target_hyp':target_hyps})
tmp = ds.train_test_split(test_size=0.01)
raw_train_dataset = tmp['train']
raw_test_dataset = tmp['test']

In [5]:
raw_test_dataset

Dataset({
    features: ['state_pp', 'target_hyp'],
    num_rows: 30438
})

In [9]:
# import torch
# from torch.utils.data import Dataset as TorchDataset, DataLoader, random_split
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW
# from tqdm import tqdm
# import os

# tokenizer = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")

# def tokenize_function(example):
#     instruction = example['state_pp']
#     response = example['target_hyp']

#     # Tokenize the instruction and response separately
#     instruction_tokens = tokenizer(instruction, padding="max_length", truncation=True, max_length=512)
#     response_tokens = tokenizer(response, padding="max_length", truncation=True, max_length=512)

#     # Combine the instruction and response into a single input
#     input_ids = instruction_tokens['input_ids']

#     # The labels are the same as the input_ids for causal language modeling
#     labels = response_tokens['input_ids']

#     return {'input_ids': input_ids, 'labels': labels}

# print("Mapping test dataset ...")
# tokenized_test = raw_test_dataset.map(tokenize_function, batched=True, num_proc=1, desc="Tokenizing dataset")
# print("Mapping train dataset ...")
# tokenized_train = raw_train_dataset.map(tokenize_function, batched=True, num_proc=6, desc="Tokenizing dataset")
# print("Done")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Mapping test dataset ...


Tokenizing dataset:   0%|          | 0/30438 [00:00<?, ? examples/s]

Mapping train dataset ...


Tokenizing dataset (num_proc=6):   0%|          | 0/3013264 [00:00<?, ? examples/s]

Done


In [22]:
# def evaluate(model, dataloader, device):
#     model.eval()
#     total_loss = 0
#     with torch.no_grad():
#         for batch in dataloader:
#             input_ids = batch["input_ids"].to(device)
#             attention_mask = batch["attention_mask"].to(device)
#             labels = batch["labels"].to(device)

#             outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#             loss = outputs.loss
#             total_loss += loss.item()

#     return total_loss / len(dataloader)

# Initialize tokenizer and model
model_name = "google/byt5-small"  # You can change this to any other Seq2Seq model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Set up optimizer
#optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

In [23]:
from transformers import Trainer, TrainingArguments
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="datasets/byt5-small",
    evaluation_strategy="steps", #"epochs"
    learning_rate=2e-5,  # PAY ATTENTION TO LEARNING RATE!
    weight_decay=0.01,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=4,
    fp16=False,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
)

#cp_path = 'gpt-neo-350m-202310/checkpoint-525000'

trainer.train()

/home/mcwave/anaconda3/envs/atp/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
1000,0.000000,nan
2000,0.000000,nan



KeyboardInterrupt



In [32]:
# num_epochs = 4
# for epoch in range(num_epochs):
#     model.train()
#     total_loss = 0

#     for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)
#         labels = batch["labels"].to(device)

#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         total_loss += loss.item()

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
    
#     # Save the model after each epoch
#     save_path = f"/home/mcwave/code/axiomatization/datasets/hyp_pred_byt5_small/trained_model_epoch_{epoch + 1}"
#     os.makedirs(save_path, exist_ok=True)
#     model.save_pretrained(save_path)
    
#     avg_train_loss = total_loss / len(train_dataloader)
#     avg_test_loss = evaluate(model, test_dataloader, device)

#     print(f"Epoch {epoch + 1}/{num_epochs}")
#     print(f"Average Train Loss: {avg_train_loss:.4f}")
#     print(f"Average Test Loss: {avg_test_loss:.4f}")

Epoch 1/4:  36%|███████████████████████████▌                                                 | 106651/298283 [21:38:44<38:53:35,  1.37it/s]


KeyboardInterrupt: 

In [36]:
avg_test_loss = evaluate(model, test_dataloader, device)

In [35]:
# save_path = f"/home/mcwave/code/axiomatization/datasets/hyp_pred_byt5_small/trained_model_epoch_{epoch + 1}"
# print(save_path)
# os.makedirs(save_path, exist_ok=True)
# model.save_pretrained(save_path)

/home/mcwave/code/axiomatization/datasets/hyp_pred_byt5_small/trained_model_epoch_1


In [6]:
save_path = f"/home/mcwave/code/axiomatization/datasets/hyp_pred_byt5_small/trained_model_epoch_1"
model = AutoModelForSeq2SeqLM.from_pretrained(save_path)

In [21]:
test_case = tokenized_test[7]
input_ids = torch.tensor(test_case['input_ids']).unsqueeze(0).to('cuda')  # Add batch dimension
print("inputs:", tokenizer.decode(test_case['input_ids'], skip_special_tokens=True))
labels = torch.tensor(test_case['labels']).to('cuda')
print("labels:", tokenizer.decode(labels, skip_special_tokens=True))

# Generate output
with torch.no_grad():
    outputs = model.generate(input_ids)

# Decode the generated output and the true labels
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
true_text = tokenizer.decode(labels, skip_special_tokens=True)

# Compare the results
print("Generated:", generated_text)
print("True:", true_text)

inputs: theorem Function.Exact.apply_apply_eq_zero (R: Type u_1) (M: Type u_2) (M': Type u_3) (N: Type u_4) (N': Type u_5) (P: Type u_6) (P': Type u_7) (f: M → N) (g: N → P) (inst✝: Zero P) (x: M) : ¬¬¬¬¬Exact f g :=
labels: ['¬¬¬¬¬g (f x) = 0']


/home/mcwave/anaconda3/envs/atp/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated: _ (S__u_7)) (G) (R
True: ['¬¬¬¬¬g (f x) = 0']


In [10]:
labels

tensor([94, 42, 74, 42, 96,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 